In [4]:
import itertools
import sys
from pathlib import Path
from collections import defaultdict

from generate_all_functions import BinaryFunction

In [5]:
inputs = 3
outputs = 3
basis = 'aig'
circuits_dir = Path('/home/vsevolod/sat/circuit_improvement/aig_res_approx')

In [6]:
grouped_functions = BinaryFunction.all_functions_grouped(inputs, outputs, basis)

In [7]:
len(grouped_functions.keys())

8886

In [8]:
# for key, values in grouped_functions.items():
#     print(f"{key}: {values}")

In [9]:
all_functions = set(BinaryFunction.all_functions(inputs, outputs))

In [10]:
len(all_functions)

2763520

In [11]:
from math import factorial

def c(n, k):
    assert 0 <= k <= n
    return factorial(n) // factorial(k) // factorial(n - k)

In [12]:
assert len(all_functions) == c(2 ** (2 ** inputs), outputs)

In [13]:
get_class = {value: key for key, values in grouped_functions.items() for value in values}

In [14]:
len(get_class.keys())

2828544

In [15]:
def check_availability(function, circuits_dir):
    assert circuits_dir.exists()
    filename = '_'.join(function.truth_tables) + '.txt'
    filepath = circuits_dir / filename
    return filepath.exists()

In [16]:
bad_set = dict()
for function in all_functions:
    normalized = get_class[function]
    if not check_availability(normalized, circuits_dir):
        bad_set[function] = normalized

In [17]:
len(bad_set)

0

In [18]:
def get_function_size(function, circuits_dir: Path) -> int:
    assert circuits_dir.exists()
    filename = '_'.join(function.truth_tables) + '.txt'
    filepath = circuits_dir / filename
    if not filepath.exists():
        return -1  # Function not found
    with filepath.open('r') as file:
        lines = file.readlines()
        function_size = len(lines) - 2
        assert function_size >= 0
        return function_size

In [19]:
classes = defaultdict(set)
functions = defaultdict(int)

for i, function in enumerate(all_functions):
    class_representative = get_class[function]
    function_size = get_function_size(class_representative, circuits_dir)
    classes[function_size].add(class_representative)
    functions[function_size] += 1
    
    if i % 100000 == 0 and i > 0:
        print(f'============{i}============', file=sys.stderr)
        for s in sorted(classes.keys()):
            print(f'{s} -> {len(classes[s])}, {functions[s]}', file=sys.stderr)
        print(f'============{i}============', file=sys.stderr)

print("Final table:")
for s in sorted(classes.keys()):
    print(f'{s} -> {len(classes[s])}, {functions[s]}')

============100000============
3 -> 50, 438
4 -> 231, 2536
5 -> 723, 8087
6 -> 1540, 18142
7 -> 2311, 27410
8 -> 2153, 26422
9 -> 1518, 14503
10 -> 222, 2421
11 -> 6, 42
============100000============
============200000============
3 -> 51, 866
4 -> 231, 5122
5 -> 725, 16192
6 -> 1540, 36267
7 -> 2316, 54853
8 -> 2161, 52694
9 -> 1587, 28945
10 -> 224, 4983
11 -> 6, 79
============200000============
============300000============
3 -> 51, 1274
4 -> 231, 7714
5 -> 725, 24246
6 -> 1540, 54309
7 -> 2316, 82166
8 -> 2163, 79265
9 -> 1606, 43387
10 -> 224, 7516
11 -> 6, 124
============300000============
============400000============
3 -> 51, 1735
4 -> 231, 10374
5 -> 726, 32422
6 -> 1540, 72457
7 -> 2317, 109622
8 -> 2163, 105427
9 -> 1616, 57698
10 -> 224, 10100
11 -> 6, 166
============400000============
============500000============
3 -> 51, 2142
4 -> 232, 13013
5 -> 726, 40641
6 -> 1540, 90642
7 -> 2318, 137108
8 -> 2163, 131509
9 -> 1619, 72087
10 -> 224, 12651
11 -> 6, 208
========

Final table:
3 -> 51, 11840
4 -> 232, 72264
5 -> 726, 223640
6 -> 1540, 498720
7 -> 2318, 762128
8 -> 2163, 727264
9 -> 1626, 396880
10 -> 224, 69664
11 -> 6, 1120


In [20]:
expected_functions = c(2 ** (2 ** inputs), outputs)
sanity_check = sum(functions.values()) == expected_functions
print(f"Sanity check: {sanity_check}")
print(f"Total functions enumerated: {sum(functions.values())}")
print(f"{expected_functions} functions expected")

Sanity check: True
Total functions enumerated: 2763520
2763520 functions expected


In [21]:
print(f"classes found rate: {1 - len(classes[-1]) / sum(map(len, classes.values()))}")
print(f"functions found rate: {1 - functions[-1] / sum(functions.values())}")

classes found rate: 1.0
functions found rate: 1.0
